In [1]:
# !pip install wheel
!pip install pyWinhook

  Using cached pyWinhook-1.6.2-cp38-cp38-win_amd64.whl (30 kB)


You should consider upgrading via the 'd:\study\4thyear_1stterm\comp sec\biometric\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [12]:
!pip install pandas
!pip install scipy
!pip install sklearn

You should consider upgrading via the 'd:\study\4thyear_1stterm\comp sec\biometric\venv\scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'd:\study\4thyear_1stterm\comp sec\biometric\venv\scripts\python.exe -m pip install --upgrade pip' command.



  Using cached sklearn-0.0.tar.gz (1.1 kB)
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


You should consider upgrading via the 'd:\study\4thyear_1stterm\comp sec\biometric\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
import pyWinhook as pyHook
import pythoncom
import os
# import matplotlib.pyplot as plt
import json
import numpy as np
import sys
import pandas as pd
from IPython.display import clear_output
import csv

In [7]:
global userName
userFilePath = "Collecting_keyStorke.csv"

In [62]:
class KeyLogger:
    def __init__(self):
        self.enterPressed = False
        self.eventList = []
        self.isCaps = False
        #self.message = ""
        
    def keyDownEvent(self, event):
        if event.KeyID == 20 and self.isCaps == False:
            self.isCaps = True
        elif event.KeyID == 20 and self.isCaps == True:
            self.isCaps = False     
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        self.storeEvent("Down", event) 
        return True
        # Fixes Requires Integer Bug (Got Nonetype)

    def keyUpEvent(self, event): 
        if event.KeyID>= 48 and event.KeyID<=57:
            event.Ascii = event.KeyID
        if self.isCaps == True and event.Ascii>=97 and event.Ascii<=122:
            event.Ascii = event.KeyID
        print(chr(event.Ascii),end='')
        self.storeEvent("Up", event)
        return True

    def mainLoop(self):
        while not self.enterPressed:
            pythoncom.PumpWaitingMessages()

    def storeEvent(self, activity, event):
        keystrokeTime = int(event.Time)
        self.eventList.append ((userName,event.Ascii,activity, int(keystrokeTime)))

        # Chosen to use Escape key (ESC) due to input using a similar method
        # Enter Key - KeyCode: 13 Ascii: 13 ScanCode: 28 - ESC = 27 @ Ascii
        if event.Ascii == 27:
            self.enterPressed = True
            userRecordData(self.eventList)

def userRecordData(eventList):
    print("\nouput")
    print(eventList)
    with open(userFilePath,'w',newline='\n') as f:
        writer = csv.writer(f)
        writer.writerow(['user', 'key', 'keyEvent', 'Time'])
        writer.writerows(eventList)
    f.close()   
    
def getUserName():
    global userName
    userName = input("Enter your Name: ")

def getKeyStroke():
    
    keyLogger = KeyLogger()
    hookManager = pyHook.HookManager()
    hookManager.KeyDown = keyLogger.keyDownEvent
    hookManager.KeyUp = keyLogger.keyUpEvent
    hookManager.HookKeyboard()

    keyLogger.mainLoop()
    # Unhooks the keyboard, no more data recorded, returns to menu
    hookManager.UnhookKeyboard()
    
getUserName()
print("Enter your text: ")
getKeyStroke()

Enter your Name: TestBoi
Enter your text: 
smoky the fire waffle bear
ouput
[('TestBoi', 13, 'Up', 17960140), ('TestBoi', 115, 'Down', 17961375), ('TestBoi', 115, 'Up', 17961453), ('TestBoi', 109, 'Down', 17961546), ('TestBoi', 111, 'Down', 17961625), ('TestBoi', 109, 'Up', 17961656), ('TestBoi', 111, 'Up', 17961703), ('TestBoi', 107, 'Down', 17962171), ('TestBoi', 107, 'Up', 17962250), ('TestBoi', 121, 'Down', 17962359), ('TestBoi', 121, 'Up', 17962437), ('TestBoi', 32, 'Down', 17962562), ('TestBoi', 32, 'Up', 17962656), ('TestBoi', 116, 'Down', 17962765), ('TestBoi', 116, 'Up', 17962843), ('TestBoi', 104, 'Down', 17962984), ('TestBoi', 104, 'Up', 17963046), ('TestBoi', 101, 'Down', 17963109), ('TestBoi', 101, 'Up', 17963203), ('TestBoi', 32, 'Down', 17963281), ('TestBoi', 32, 'Up', 17963359), ('TestBoi', 102, 'Down', 17963468), ('TestBoi', 102, 'Up', 17963562), ('TestBoi', 105, 'Down', 17963593), ('TestBoi', 105, 'Up', 17963671), ('TestBoi', 114, 'Down', 17963703), ('TestBoi', 101, '

In [19]:
import pandas as pd
import numpy as np
import csv
from scipy.spatial.distance import cityblock
from sklearn.metrics import roc_curve

In [63]:
data = pd.read_csv("Collecting_keyStorke.csv")
userList = data.user.unique()
keyList = data.key.unique()
df = pd.DataFrame(columns=['subject','key','DD'])
for i in range(0, len(userList)):
    for j in range(0,len(keyList)):
        queryData = data.query("user=='" +userList[i]+ "' and key==" + str(keyList[j]) + " and key >=33 and key<=122")
        queryLen = len(queryData)
        finalData = {}
        if queryLen > 0:
            if(queryLen > 2):
                for k in range(0,queryLen,2):
                    finalData['subject'] = userList[i]
                    keyUpIndex = queryData.iloc[k+1].name
                    finalData['key'] = chr(keyList[j]) + ' ' + chr(keyList[j + 1])
                    if(data.iloc[keyUpIndex + 1].user == userList[i]):
                        finalData['DD'] = (int(data.iloc[keyUpIndex+1].Time) - int(queryData.iloc[k].Time))/1000
                    df = df.append(finalData,ignore_index=True )
            else:
                finalData['subject'] = userList[i]
                keyUpIndex = queryData.query("keyEvent=='Up'").index[0]
                finalData['key'] = chr(keyList[j]) + ' ' + chr(keyList[j + 1])
                if(data.iloc[keyUpIndex + 1].user == userList[i]):
                        finalData['DD'] = (int(data.iloc[keyUpIndex+1].Time) - int( queryData.query("keyEvent=='Down'").Time))/1000
                df = df.append(finalData,ignore_index=True )
            
           
                
f = open("KeyStrokeDistance.csv", 'a', newline='\n')
writer = csv.writer(f)
groupedDf = df.groupby(['subject','key']).mean().reset_index()
# writer.writerow(['subject','key','DD'])
for row in groupedDf.iterrows():
#     print(row[1])
    writer.writerow(row[1])
    
f.close()
print(groupedDf[groupedDf.subject == 'John'])

Empty DataFrame
Columns: [subject, key, DD]
Index: []


In [64]:
groupedDf.pivot(index="subject", columns="key", values="DD").to_csv('pivoted.csv')

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def load_data(file_name, col_to_remove):
    df = pd.read_csv(file_name)

    DD_columns = [col for col in df.columns if col.startswith('DD')]

    data = {}
    data['total'] = df.drop(columns=col_to_remove)
    data['DD']    = df[DD_columns]
    data['pca3']  = pd.DataFrame(PCA(n_components=3).fit_transform(data['total']))
   # data['pca10'] = pd.DataFrame(PCA(n_components=10).fit_transform(data['total']))
    
    return data, df['subject'].values

def calculate_KMeans(data,y):
    X = data['total']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=y)
    n_neighbors = [2, 3, 4, 5, 6, 7, 8, 9, 10]
    parameters = dict(n_neighbors=n_neighbors)
    clf = KNeighborsClassifier()
    grid = GridSearchCV(clf, parameters, cv=5)
    grid.fit(X_train, Y_train)
    results = grid.cv_results_

    Y_pred = grid.predict(X_test)

    fpr, tpr, threshold = roc_curve(Y_test.ravel(), Y_pred.ravel())

    return fpr, tpr, threshold

In [57]:
df = pd.read_csv('pivoted copy.csv')
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(df[df.columns[1:]].to_numpy(), [7, 13])

KNeighborsClassifier(n_neighbors=1)

In [65]:
df = pd.read_csv('pivoted.csv')
inp = df[df.columns[1:]].to_numpy()[0]
clf.predict([inp])

array([7])

array([0.2575    , 0.109     , 0.2265    , 0.18733333, 0.109     ,
       0.11      , 0.141     , 0.516     , 0.172     , 0.484     ,
       0.5545    , 0.281     , 0.219     , 0.218     , 0.515     ])